In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics

import os, sys
from time import time

from phm08ds.models import experiment

## Load Dataset

In [2]:
folderpath = '../../../data/interim/'
data_op_02 = pd.read_csv(folderpath + 'data_op_02.csv')
data_op_02.head()

,unit,time_step,operational_setting_1,operational_setting_2,operational_setting_3,Sensor_2,Sensor_3,Sensor_4,Sensor_7,Sensor_11,Sensor_12,Sensor_15,Operational_condition,Health_state
0,1,1,10.0047,0.2501,20.0,1499.45,1309.95,10.52,2318.87,372.15,2388.13,0.03,2,1
14,1,15,9.9988,0.2500,20.0,1497.87,1302.51,10.52,2318.90,371.84,2388.12,0.03,2,1
15,1,16,9.9987,0.2500,20.0,1500.47,1311.39,10.52,2318.87,372.19,2388.08,0.03,2,1
17,1,18,10.0066,0.2507,20.0,1502.57,1304.56,10.52,2318.93,372.03,2388.06,0.03,2,1
35,1,36,10.0024,0.2504,20.0,1499.85,1306.89,10.52,2318.94,371.77,2388.11,0.03,2,2


## Data preprocessing

### Get rid of informations there are not sensor readings

Wang (2008) reports Sensor 15 has importat information. However, there are no relevant informations of this sensor. The data seems to be corrupted like this:

Let's remove it from our database creating an object transformer.

In [3]:
from phm08ds.features.feature_selection import RemoveSensor

tf_remove_sensor_15 = RemoveSensor(sensors=[15])
data_op_02 = tf_remove_sensor_15.fit_transform(data_op_02)
data_op_02.head()

,unit,time_step,operational_setting_1,operational_setting_2,operational_setting_3,Sensor_2,Sensor_3,Sensor_4,Sensor_7,Sensor_11,Sensor_12,Operational_condition,Health_state
0,1,1,10.0047,0.2501,20.0,1499.45,1309.95,10.52,2318.87,372.15,2388.13,2,1
14,1,15,9.9988,0.2500,20.0,1497.87,1302.51,10.52,2318.90,371.84,2388.12,2,1
15,1,16,9.9987,0.2500,20.0,1500.47,1311.39,10.52,2318.87,372.19,2388.08,2,1
17,1,18,10.0066,0.2507,20.0,1502.57,1304.56,10.52,2318.93,372.03,2388.06,2,1
35,1,36,10.0024,0.2504,20.0,1499.85,1306.89,10.52,2318.94,371.77,2388.11,2,2


Before feeding to the classifier, let's remove unwanted information, such as unit, time_step and operational settings.

In [4]:
from phm08ds.features.feature_selection import RemoveInfo

tf_remove_info = RemoveInfo()

data_with_features = tf_remove_info.fit_transform(data_op_02)
data_with_features.head()

,Sensor_2,Sensor_3,Sensor_4,Sensor_7,Sensor_11,Sensor_12,Health_state
0,1499.45,1309.95,10.52,2318.87,372.15,2388.13,1
14,1497.87,1302.51,10.52,2318.90,371.84,2388.12,1
15,1500.47,1311.39,10.52,2318.87,372.19,2388.08,1
17,1502.57,1304.56,10.52,2318.93,372.03,2388.06,1
35,1499.85,1306.89,10.52,2318.94,371.77,2388.11,2


We need to normalize our data. Let's use Z-score standardization.

In [5]:
from sklearn.preprocessing import StandardScaler

tf_std_scaller = preprocessing.StandardScaler()
data_with_features_std = tf_std_scaller.fit_transform(data_with_features.drop(labels='Health_state', axis=1))
data_with_features_std

array([[-0.46078266, -0.16893857,  1.        , -0.97980055,  1.35615577,
        -0.3602389 ],
       [-0.73264726, -1.06544604,  1.        , -0.65864984,  0.76511402,
        -0.46521261],
       [-0.28527512,  0.00457901,  1.        , -0.97980055,  1.43241922,
        -0.88510744],
       ...,
       [ 1.3837671 ,  1.64094613,  1.        ,  1.58940513, -1.4846578 ,
         1.00441932],
       [ 0.99833879,  1.8867627 ,  1.        ,  1.69645537, -1.80877746,
         2.36907754],
       [ 0.44772693,  1.865073  ,  1.        ,  1.4823549 , -2.07569954,
         1.9491827 ]])

In [6]:
labels = np.array(data_with_features['Health_state'])
labels

array([1, 1, 1, ..., 4, 4, 4])

# Classification steps

## Load Experiment model

In [7]:
from phm08ds.models import experiment

## Define classifiers and its specifications

In [8]:
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

In [9]:
# MLP:
mlp_clf = MLPClassifier(solver='adam', learning_rate='adaptive', 
                        max_iter=1300, learning_rate_init=5e-04, tol=1e-4)

## Put all clf in a dictionary:

In [10]:
classifiers = {'MLP': mlp_clf}

Since we are using SVM and MLP we need to extract all power from those methods. Let's perform a Random Search to parameters optimizations.

### Hyperparameter tunning

In [11]:
from sklearn.model_selection import RandomizedSearchCV

random_search = dict((k,[]) for k in classifiers.keys())

In [12]:
param_dist_dict = {'MLP': {"hidden_layer_sizes": list(np.arange(2,1001))}
                  }

In [13]:
for clf in param_dist_dict.keys():
    start = time()
    random_search[clf] = RandomizedSearchCV(classifiers[clf], param_dist_dict[clf], cv=10, n_iter=50, verbose=5, n_jobs=100, scoring='accuracy')
    random_search[clf].fit(data_with_features_std, labels)
    print('Elapsed time:')
    print(time() - start)

Fitting 10 folds for each of 50 candidates, totalling 500 fits
[CV] hidden_layer_sizes=433 ..........................................
[CV] hidden_layer_sizes=433 ..........................................
[CV] hidden_layer_sizes=433 ..........................................
[CV] hidden_layer_sizes=433 ..........................................
[CV] hidden_layer_sizes=433 ..........................................
[CV] hidden_layer_sizes=433 ..........................................
[CV] hidden_layer_sizes=433 ..........................................
[CV] hidden_layer_sizes=433 ..........................................
[CV] hidden_layer_sizes=433 ..........................................
[CV] hidden_layer_sizes=581 ..........................................
[CV] hidden_layer_sizes=581 ..........................................
[CV] hidden_layer_sizes=581 ..........................................
[CV] hidden_layer_sizes=581 ..........................................
[CV] hidden_la

[Parallel(n_jobs=100)]: Done  88 tasks      | elapsed: 16.0min


[CV] hidden_layer_sizes=351 ..........................................
[CV] hidden_layer_sizes=351 ..........................................
[CV] .............. hidden_layer_sizes=584, score=0.625, total=16.1min
[CV] hidden_layer_sizes=351 ..........................................
[CV] . hidden_layer_sizes=104, score=0.6005830903790087, total= 9.5min
[CV] hidden_layer_sizes=351 ..........................................
[CV] . hidden_layer_sizes=584, score=0.6468023255813954, total=16.3min
[CV] hidden_layer_sizes=81 ...........................................
[CV] . hidden_layer_sizes=104, score=0.6355685131195336, total=10.0min
[CV] .............. hidden_layer_sizes=475, score=0.625, total=16.6min
[CV] hidden_layer_sizes=81 ...........................................
[CV] hidden_layer_sizes=81 ...........................................
[CV] . hidden_layer_sizes=581, score=0.6153846153846154, total=16.6min
[CV] hidden_layer_sizes=81 ...........................................
[CV] .

[Parallel(n_jobs=100)]: Done 250 tasks      | elapsed: 43.7min


[CV] . hidden_layer_sizes=156, score=0.6191860465116279, total=12.1min
[CV] hidden_layer_sizes=509 ..........................................
[CV] . hidden_layer_sizes=312, score=0.5869565217391305, total=12.1min
[CV] hidden_layer_sizes=509 ..........................................
[CV] . hidden_layer_sizes=943, score=0.6260869565217392, total=17.9min
[CV] hidden_layer_sizes=509 ..........................................
[CV] . hidden_layer_sizes=342, score=0.6240928882438317, total=16.9min
[CV] hidden_layer_sizes=509 ..........................................
[CV] . hidden_layer_sizes=342, score=0.6240928882438317, total=17.5min
[CV] hidden_layer_sizes=509 ..........................................
[CV] . hidden_layer_sizes=312, score=0.6322674418604651, total=12.7min
[CV] hidden_layer_sizes=509 ..........................................
[CV] . hidden_layer_sizes=342, score=0.6034985422740525, total=17.6min
[CV] . hidden_layer_sizes=312, score=0.6468023255813954, total=13.2min
[CV] h

[Parallel(n_jobs=100)]: Done 402 out of 500 | elapsed: 81.8min remaining: 19.9min


[CV] . hidden_layer_sizes=334, score=0.6311953352769679, total=11.9min
[CV] . hidden_layer_sizes=799, score=0.6540697674418605, total=43.7min
[CV] .. hidden_layer_sizes=238, score=0.613933236574746, total=21.3min
[CV] . hidden_layer_sizes=799, score=0.5985507246376811, total=44.4min
[CV] . hidden_layer_sizes=238, score=0.6337209302325582, total=22.1min
[CV] . hidden_layer_sizes=334, score=0.6342525399129173, total=15.2min
[CV] .. hidden_layer_sizes=769, score=0.636231884057971, total=30.5min
[CV] . hidden_layer_sizes=637, score=0.6226415094339622, total=27.8min
[CV] .. hidden_layer_sizes=637, score=0.597667638483965, total=26.4min
[CV] .. hidden_layer_sizes=997, score=0.626453488372093, total=55.9min
[CV] .. hidden_layer_sizes=799, score=0.626453488372093, total=46.8min
[CV] . hidden_layer_sizes=769, score=0.6386066763425254, total=31.8min
[CV] . hidden_layer_sizes=601, score=0.6293604651162791, total=36.5min
[CV] . hidden_layer_sizes=334, score=0.6322674418604651, total=17.5min
[CV] .

[Parallel(n_jobs=100)]: Done 500 out of 500 | elapsed: 95.9min finished


Elapsed time:
5778.133532524109


## Savel results, models and pipeline to a .pkl file 

In [14]:
from sklearn.pipeline import Pipeline

data_preprocessing = Pipeline([('remove_sensor_15', tf_remove_sensor_15),
                               ('remove_info', tf_remove_info),
                               ('std_scaler', tf_std_scaller)
                              ])

In [15]:
experiment.save_models(random_search)
experiment.save_pipeline(data_preprocessing)